In [1]:
from __future__ import annotations


from oqd_compiler_infrastructure import TypeReflectBaseModel

In [2]:
class MyProgram(TypeReflectBaseModel):
    expr: MyExpr

In [3]:
class MyExpr(TypeReflectBaseModel):
    pass

In [4]:
class MyInt(MyExpr):
    value: int

In [5]:
class MyAdd(MyExpr):
    left: MyExpr
    right: MyExpr

In [6]:
class MyMul(MyExpr):
    left: MyExpr
    right: MyExpr

In [7]:
class MyPow(MyExpr):
    left: MyExpr
    right: MyExpr

In [15]:
prog = MyProgram(expr=MyAdd(left=MyInt(value=1), right=MyInt(value=2)))

prog

MyProgram(class_='MyProgram', expr=MyAdd(class_='MyAdd', left=MyInt(class_='MyInt', value=1), right=MyInt(class_='MyInt', value=2)))